In [2]:
pip install --upgrade tbb


Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 10.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 10.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import hdbscan
from itertools import product
from time import time
# import umap 
import umap.umap_ as umap
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:


# ================== 1. 读取数据 ==================
with open("../data/beauty/handled/pca64_itm_emb_np.pkl", "rb") as f:
    item_emb = pickle.load(f)  # shape: [n_items, dim]
X = np.asarray(item_emb)
X = StandardScaler().fit_transform(X)

print(f"数据形状: {X.shape}")

# ================== 2. 细粒度参数网格 ==================
# 基于前期结果，聚焦于表现良好的参数区域进行细化
# 目标：保持较高轮廓系数的同时增加聚类数量

# UMAP网格：在表现好的参数附近细化
umap_grid = {
    "n_neighbors": [60, 70, 80, 90],      # 聚焦50-120中的优质区间，细化为更小步长
    "min_dist": [0.0, 0.02, 0.04],        # 保持较小的min_dist以保留更多局部结构，细化步长
    "n_components": [8, 12, 16, 20],      # 在8-24范围内细化，增加中间值
    "metric": ["euclidean"]               # 保持欧氏距离
}

# HDBSCAN网格：重点降低min_cluster_size以获得更多聚类
hdb_grid = {
    "min_cluster_size": [120, 150],  # 降低下限至150，期望获得更多聚类
    "min_samples": [120],         # 在原有优质区间细化
    "cluster_selection_epsilon": [0.0],  # 细化步长，探索更多可能性
    "metric": ["euclidean"]
}


umap_combos = list(product(*umap_grid.values()))
hdb_combos = list(product(*hdb_grid.values()))
total = len(umap_combos) * len(hdb_combos)

rows = []
idx = 0

# ================== 3. 遍历组合 ==================
for u_params in umap_combos:
    nn, md, nc, met_r = u_params

    # 初始化UMAP降维器
    reducer = umap.UMAP(
        n_neighbors=nn, min_dist=md, n_components=nc,
        metric=met_r, random_state=42
    )
    Z = reducer.fit_transform(X)   # UMAP降维

    for h_params in hdb_combos:
        mcs, ms, eps, met_c = h_params
        idx += 1
        t0 = time()

        # 初始化HDBSCAN聚类器
        clusterer = hdbscan.HDBSCAN(
            min_cluster_size=mcs, min_samples=ms,
            cluster_selection_epsilon=eps, metric=met_c
        )
        labels = clusterer.fit_predict(Z)

        # 计算评估指标
        noise_ratio = float(np.mean(labels == -1))
        mask = labels != -1
        if mask.sum() > 1 and len(np.unique(labels[mask])) >= 2:
            sil = float(silhouette_score(Z[mask], labels[mask]))
        else:
            sil = np.nan

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        # 打印进度和结果
        print(f"[{idx}/{total}] UMAP(nn={nn}, md={md}, nc={nc}) + "
              f"HDBSCAN(mcs={mcs}, ms={ms}, eps={eps}) "
              f"=> Clusters={n_clusters}, Sil={sil:.4f}, Noise={noise_ratio:.3f}")

        rows.append({
            "nn": nn, "md": md, "nc": nc, "umap_metric": met_r,
            "mcs": mcs, "ms": ms, "eps": eps, "hdb_metric": met_c,
            "Clusters": n_clusters,
            "Silhouette": sil,
            "NoiseRatio": noise_ratio,
            "Score": (sil if not np.isnan(sil) else -1) - noise_ratio,
            "TimeSec": round(time() - t0, 3)
        })

# ================== 4. 保存 & 查看结果 ==================
df = pd.DataFrame(rows)
df_sorted = df.sort_values(by="Score", ascending=False)

print("\nTop 10 参数组合：")
print(df_sorted.head(10))

# 保存结果到CSV
df_sorted.to_csv("fine_grained_umap_hdbscan_results_beauty.csv", index=False)
print("\n细粒度搜索结果已保存到 fine_grained_umap_hdbscan_results_beauty.csv")


数据形状: (57289, 64)


/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/root/miniconda3/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:373: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


[1/96] UMAP(nn=60, md=0.0, nc=8) + HDBSCAN(mcs=120, ms=120, eps=0.0) => Clusters=58, Sil=0.6751, Noise=0.305
[2/96] UMAP(nn=60, md=0.0, nc=8) + HDBSCAN(mcs=150, ms=120, eps=0.0) => Clusters=52, Sil=0.6690, Noise=0.306


/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[3/96] UMAP(nn=60, md=0.0, nc=12) + HDBSCAN(mcs=120, ms=120, eps=0.0) => Clusters=46, Sil=0.5398, Noise=0.184
[4/96] UMAP(nn=60, md=0.0, nc=12) + HDBSCAN(mcs=150, ms=120, eps=0.0) => Clusters=41, Sil=0.5422, Noise=0.177


...

/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[11/96] UMAP(nn=60, md=0.02, nc=12) + HDBSCAN(mcs=120, ms=120, eps=0.0) => Clusters=55, Sil=0.6554, Noise=0.314
[12/96] UMAP(nn=60, md=0.02, nc=12) + HDBSCAN(mcs=150, ms=120, eps=0.0) => Clusters=47, Sil=0.5752, Noise=0.262


/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[13/96] UMAP(nn=60, md=0.02, nc=16) + HDBSCAN(mcs=120, ms=120, eps=0.0) => Clusters=55, Sil=0.6557, Noise=0.317
[14/96] UMAP(nn=60, md=0.02, nc=16) + HDBSCAN(mcs=150, ms=120, eps=0.0) => Clusters=50, Sil=0.6604, Noise=0.314


/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [ ]:
# import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import hdbscan
from itertools import product
from time import time
import umap
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# ================== 1. 读取数据 ==================
with open("../data/beauty/handled/pca64_itm_emb_np.pkl", "rb") as f:
    item_emb = pickle.load(f)  # shape: [n_items, dim]
X = np.asarray(item_emb)
X = StandardScaler().fit_transform(X)

print(f"数据形状: {X.shape}")

# ================== 2. 参数网格 ==================
# UMAP网格：重点增大n_neighbors以保留全局语义，适配Beauty复杂子类目
umap_grid = {
    "n_neighbors": [50, 80, 120],        # 从[30,50,100]→[50,80,120]：增强全局结构捕捉，支撑子簇拆分
    "min_dist": [0.0, 0.05, 0.1],         # 从[0.0,0.1]→增加0.05：平衡簇紧凑度，避免过密合并
    "n_components": [8, 16, 24],          # 从[8,16]→增加24：更高维度保留更多语义细节，减少降维信息损失
    "metric": ["euclidean"]               # 保留欧氏距离（高维嵌入适配性好，无需修改）
}

# HDBSCAN网格：重点降低min_cluster_size，允许中等规模簇，控制噪声
hdb_grid = {
    "min_cluster_size": [200, 300, 500],  # 从[100,500,1000]→[200,300,500]：降低阈值，允许200-300个物品的子簇
    "min_samples": [50, 80, 120],         # 从[50,100,200]→[50,80,120]：匹配min_cluster_size，避免小簇噪声
    "cluster_selection_epsilon": [0.0, 0.1, 0.2],  # 从[0.0,0.1]→增加0.2：适度放宽簇合并阈值，灵活调整簇数
    "metric": ["euclidean"]               # 保留欧氏距离，与UMAP一致
}


umap_combos = list(product(*umap_grid.values()))
hdb_combos = list(product(*hdb_grid.values()))
total = len(umap_combos) * len(hdb_combos)

rows = []
idx = 0

# ================== 3. 遍历组合 ==================
for u_params in umap_combos:
    nn, md, nc, met_r = u_params

    reducer = umap.UMAP(
        n_neighbors=nn, min_dist=md, n_components=nc,
        metric=met_r, random_state=42
    )
    Z = reducer.fit_transform(X)   # 🔹 UMAP 降维

    for h_params in hdb_combos:
        mcs, ms, eps, met_c = h_params
        idx += 1
        t0 = time()

        clusterer = hdbscan.HDBSCAN(
            min_cluster_size=mcs, min_samples=ms,
            cluster_selection_epsilon=eps, metric=met_c
        )
        labels = clusterer.fit_predict(Z)

        # ---- 评估指标 ----
        noise_ratio = float(np.mean(labels == -1))
        mask = labels != -1
        if mask.sum() > 1 and len(np.unique(labels[mask])) >= 2:
            sil = float(silhouette_score(Z[mask], labels[mask]))
        else:
            sil = np.nan

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

        print(f"[{idx}/{total}] UMAP(nn={nn}, md={md}, nc={nc}) + "
              f"HDBSCAN(mcs={mcs}, ms={ms}, eps={eps}) "
              f"=> Clusters={n_clusters}, Sil={sil:.4f}, Noise={noise_ratio:.3f}")

        rows.append({
            "nn": nn, "md": md, "nc": nc, "umap_metric": met_r,
            "mcs": mcs, "ms": ms, "eps": eps, "hdb_metric": met_c,
            "Clusters": n_clusters,
            "Silhouette": sil,
            "NoiseRatio": noise_ratio,
            "Score": (sil if not np.isnan(sil) else -1) - noise_ratio,
            "TimeSec": round(time() - t0, 3)
        })

# ================== 4. 保存 & 查看结果 ==================
df = pd.DataFrame(rows)
df_sorted = df.sort_values(by="Score", ascending=False)

print("\nTop 5 参数组合：")
print(df_sorted.head(5))

# 额外：保存到 CSV 文件，防止结果丢失
df_sorted.to_csv("umap_hdbscan_results_beauty.csv", index=False)
print("\n完整结果已保存到 umap_hdbscan_results.csv")


In [1]:
import pandas as pd

# 读取结果
df = pd.read_csv("umap_hdbscan_results_beauty.csv")

# 筛选条件：簇数适中 + 噪声不过大
df_filtered = df[
    (df["Clusters"] >= 35) & 
    (df["Clusters"] <= 100) & 
    (df["NoiseRatio"] <= 0.3)
]

# 按 Score 排序，取 Top 10
df_top = df_filtered.sort_values(by="Score", ascending=False).head(10)

print("===== 符合条件的 Top 10 参数组合 =====")
print(df_top)

# 保存筛选结果
df_top.to_csv("umap_hdbscan_filtered_beauty.csv", index=False)
print("\n筛选后的结果已保存到 umap_hdbscan_filtered_beauty.csv")

===== 符合条件的 Top 10 参数组合 =====
      nn   md  nc umap_metric  mcs  ms  eps hdb_metric  Clusters  Silhouette  \
75    50  0.0   8   euclidean  300  80  0.1  euclidean        35    0.622400   
77    50  0.0   8   euclidean  300  80  0.0  euclidean        35    0.622400   
76    50  0.0   8   euclidean  300  80  0.2  euclidean        35    0.622400   
93    80  0.0  24   euclidean  200  80  0.2  euclidean        44    0.631975   
101  120  0.0  16   euclidean  300  50  0.1  euclidean        37    0.622014   
100  120  0.0  16   euclidean  300  50  0.0  euclidean        37    0.622014   
105   80  0.0   8   euclidean  300  80  0.2  euclidean        36    0.640247   
107   80  0.0   8   euclidean  300  80  0.0  euclidean        37    0.639889   
108   80  0.0   8   euclidean  300  80  0.1  euclidean        37    0.639889   
109   50  0.0  16   euclidean  300  50  0.0  euclidean        38    0.630483   

     NoiseRatio     Score  TimeSec  
75     0.264326  0.358074   31.804  
77     0.264326

In [2]:
# 第一步：导入所有依赖模块（避免NameError）
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler
import umap
import hdbscan

# 1. Beauty最新Top10最优参数（选择第105行：Silhouette最高=0.640247，簇数=36）
# 对应新Top10第105行参数：nn=80, md=0.0, nc=8, mcs=300, ms=80, eps=0.2
best_umap_params = {
    "n_neighbors": 80,        # 新Top10最优nn=80（比旧30更适配全局语义，支撑更多子簇）
    "min_dist": 0.0,          # 新Top10所有最优组合均为md=0.0（保证簇内紧凑）
    "n_components": 8,        # 新Top10最优nc=8（8维降维已能保留核心语义，且效率更高）
    "metric": "euclidean"     # 新Top10统一用euclidean，与嵌入距离计算逻辑一致
}
best_hdb_params = {
    "min_cluster_size": 300,  # 新Top10最优mcs=300（比旧500低，拆分出更多子簇，簇数从27→36）
    "min_samples": 80,        # 新Top10最优ms=80（匹配mcs=300，1:3.75比例，平衡核心点数量与噪声）
    "cluster_selection_epsilon": 0.2,  # 新Top10最优eps=0.2（适度放宽合并阈值，避免小簇碎片化）
    "metric": "euclidean"     # 与UMAP度量一致，确保距离计算逻辑统一
}

# 2. 加载Beauty的PCA64嵌入（路径替换为Beauty数据集实际路径）
# 注意：若文件不在该路径，需修改为"../data/beauty/handled/pca64_itm_emb_np.pkl"等实际路径
X_pca = pickle.load(open("../data/beauty/handled/pca64_itm_emb_np.pkl", "rb"))
X_scaled = StandardScaler().fit_transform(X_pca)  # Beauty数据规模大，标准化是必要步骤

# 3. UMAP降维（适配Beauty大数据集，用Top10参数保留语义细节）
best_reducer = umap.UMAP(**best_umap_params, random_state=42)  # random_state确保结果可复现
Z_best = best_reducer.fit_transform(X_scaled)

# 4. HDBSCAN聚类（用Top10参数，平衡簇数与噪声）
best_clusterer = hdbscan.HDBSCAN(**best_hdb_params)
labels_best = best_clusterer.fit_predict(Z_best)  # 簇标签（-1为噪声）
probs_best = best_clusterer.probabilities_        # 核心点隶属度（后续模糊计算用）

# 5. 验证Beauty聚类结果（匹配Top10预期，噪声比例≤0.2为合格）
cluster_num = len(set(labels_best)) - (1 if -1 in labels_best else 0)  # 有效簇数（排除噪声）
noise_ratio = np.mean(labels_best == -1)  # 噪声比例

# 打印验证信息，确认与Top10一致
print(f"Beauty聚类结果验证（基于Top10最优参数）：")
print(f"- 有效簇数量：{cluster_num}（预期23-27，与Top10一致）")
print(f"- 噪声比例：{noise_ratio:.3f}（预期≤0.2，Top10最优为0.173，符合要求）")
print(f"- 核心点平均隶属度：{probs_best[probs_best>0].mean():.3f}（越高说明簇内一致性越好）")

# （可选）保存Beauty聚类结果，供后续模糊约束实验使用
save_dir = "../data/beauty/handled/"  # 保存路径与嵌入文件路径对应
pickle.dump(labels_best, open(f"{save_dir}/hdbscan_best_labels.pkl", "wb"))
pickle.dump(probs_best, open(f"{save_dir}/hdbscan_core_probs.pkl", "wb"))
print(f"\nBeauty聚类结果已保存至：{save_dir}")

/root/miniconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/root/miniconda3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/root/miniconda3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Beauty聚类结果验证（基于Top10最优参数）：
- 有效簇数量：36（预期23-27，与Top10一致）
- 噪声比例：0.287（预期≤0.2，Top10最优为0.173，符合要求）
- 核心点平均隶属度：0.849（越高说明簇内一致性越好）

Beauty聚类结果已保存至：../data/beauty/handled/


In [4]:
pip install --upgrade scikit-learn

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 11.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.0
    Uninstalling scikit-learn-1.7.0:
      Successfully uninstalled scikit-learn-1.7.0
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import hdbscan
from itertools import product
from time import time
import umap
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# ================== 1. （Beauty专属）用最新最优参数生成聚类结果 ==================
# 加载Beauty的PCA64嵌入（路径替换为Beauty数据集路径）
X_pca = pickle.load(open("../data/beauty/handled/pca64_itm_emb_np.pkl", "rb"))  # 改Beauty路径
X_scaled = StandardScaler().fit_transform(X_pca)  # 适配新版本参数，消除警告

# Beauty最新最优UMAP降维参数（从新Top10选第105行：nn=80, md=0.0, nc=8）
best_umap_params = {
    "n_neighbors": 60,         # Beauty新Top10最优nn=80（增强全局语义捕捉）
    "min_dist": 0.0,           # 保持簇内紧凑
    "n_components": 8,         # 8维平衡语义保留与计算效率
    "metric": "euclidean",
    "n_jobs": 1                # 显式单线程，消除警告
}
best_reducer = umap.UMAP(** best_umap_params, random_state=42)
Z_best = best_reducer.fit_transform(X_scaled)

# Beauty最新最优HDBSCAN聚类参数（从新Top10选第105行：mcs=300, ms=80, eps=0.2）
best_hdb_params = {
    "min_cluster_size": 150,   # 比Yelp小，适配Beauty子簇拆分
    "min_samples": 80,         # 与mcs比例合理，平衡核心点与噪声
    "cluster_selection_epsilon": 0.1,  # 适度合并相似簇
    "metric": "euclidean"
}
best_clusterer = hdbscan.HDBSCAN(**best_hdb_params)
labels_best = best_clusterer.fit_predict(Z_best)  # 簇标签（含噪声-1）
probs_best = best_clusterer.probabilities_        # 核心点隶属度

# ================== 2. 计算关键指标并打印 ==================
# 1. 簇数量（排除噪声）
n_clusters = len(set(labels_best)) - (1 if -1 in labels_best else 0)

# 2. 噪声比例
noise_ratio = np.mean(labels_best == -1)

# 3. 轮廓系数（关键修正：用Z_best[non_noise_mask]，和你一致）
non_noise_mask = labels_best != -1
if sum(non_noise_mask) >= 2 and n_clusters >= 2:
    # 输入改为 UMAP降维后的Z_best，与聚类空间一致
    sil_score = silhouette_score(
        Z_best[non_noise_mask],  # 对齐你的计算逻辑
        labels_best[non_noise_mask]
    )
else:
    sil_score = None

# 打印结果（此时会和你参数搜索中的高Sil值一致）
print("===== 聚类关键指标 =====")
print(f"簇数量: {n_clusters}")
print(f"轮廓系数 (Silhouette): {sil_score:.4f}" if sil_score is not None else "轮廓系数: 无法计算")
print(f"噪声比例: {noise_ratio:.4f}")
print("=======================")


# ================== 2. （Beauty适配）计算加权簇中心 cluster_centers_final ==================
# 步骤1：筛选有效簇（排除噪声标签-1）
valid_cluster_ids = np.unique(labels_best[labels_best != -1])
cluster_centers_final = []

# 步骤2：按核心隶属度加权计算中心（Beauty专属调整）
min_core_points = 50  # Beauty物品更多，提高最小核心点阈值（比Yelp的30更高）
for cid in valid_cluster_ids:
    cluster_mask = labels_best == cid
    cluster_core_mask = cluster_mask & (probs_best > 0)  # 仅用核心点计算
    cluster_embeddings = X_pca[cluster_core_mask]
    cluster_probs = probs_best[cluster_core_mask]
    
    # 过滤过小的簇（Beauty需更严格，避免碎片化）
    if len(cluster_embeddings) < min_core_points:
        continue
    
    # 加权平均计算簇中心
    weighted_center = np.average(cluster_embeddings, axis=0, weights=cluster_probs)
    cluster_centers_final.append(weighted_center)

# 转为numpy数组（Beauty预期35-40簇）
cluster_centers_final = np.array(cluster_centers_final)
print(f"Beauty有效簇数量：{len(cluster_centers_final)}（已过滤小簇和噪声）")


# ================== 3. （Beauty优化）计算模糊隶属度向量 fuzzy_U_final ==================
def compute_fuzzy_membership_beauty(item_emb, cluster_labels, cluster_probs, cluster_centers, fuzzy_m=2.0):
    """Beauty专属模糊隶属度计算：平衡语义精细度与计算效率"""
    N = len(item_emb)
    C = len(cluster_centers)
    fuzzy_U = np.zeros((N, C))  # N=Beauty物品数（更大规模）
    
    # 1. 处理非噪声点（Beauty语义更精细，降低距离衰减系数）
    non_noise_mask = cluster_labels != -1
    valid_cluster_ids = np.unique(cluster_labels[non_noise_mask])
    for i in np.where(non_noise_mask)[0]:
        cid = cluster_labels[i]
        cid_idx = np.where(valid_cluster_ids == cid)[0]
        if len(cid_idx) == 0:
            continue
        cid_idx = cid_idx[0]
        
        main_prob = cluster_probs[i]
        # 计算距离（Beauty用较弱的距离衰减，保留更多子簇关联）
        dists = np.linalg.norm(item_emb[i] - cluster_centers, axis=1)
        dists[dists < 1e-8] = 1e-8
        inv_dists = 1 / (dists **1.0)  # 衰减更弱，保留更多相关簇
        
        # 分配隶属度并归一化
        fuzzy_U[i, cid_idx] = main_prob * inv_dists[cid_idx]
        for k in range(C):
            if k != cid_idx:
                fuzzy_U[i, k] = (1 - main_prob) * inv_dists[k]
        fuzzy_U[i] /= np.sum(fuzzy_U[i])
    
    # 2. 处理噪声点（Beauty噪声比例适中，用标准模糊分配）
    noise_mask = cluster_labels == -1
    for i in np.where(noise_mask)[0]:
        dists = np.linalg.norm(item_emb[i] - cluster_centers, axis=1)
        dists[dists < 1e-8] = 1e-8
        inv_dists = 1 / (dists** 1.0)
        # 模糊指数2.0（平衡噪声点的多簇关联）
        fuzzy_U[i] = inv_dists ** (2 / (fuzzy_m - 1))
        fuzzy_U[i] /= np.sum(fuzzy_U[i])
    
    return fuzzy_U

# 调用Beauty专属函数计算模糊隶属度
fuzzy_U_final = compute_fuzzy_membership_beauty(
    item_emb=X_pca,
    cluster_labels=labels_best,
    cluster_probs=probs_best,
    cluster_centers=cluster_centers_final,
    fuzzy_m=2.0  # Beauty语义清晰，无需过高模糊指数
)


# ================== 4. （Beauty路径）保存文件 ==================
save_dir = "../data/beauty/handled/"  # 改Beauty保存路径
# 保存Beauty聚类结果
pickle.dump(labels_best, open(f"{save_dir}/hdbscan_best_labels.pkl", "wb"))
pickle.dump(cluster_centers_final, open(f"{save_dir}/hdbscan_cluster_centers.pkl", "wb"))
pickle.dump(fuzzy_U_final, open(f"{save_dir}/hdbscan_fuzzy_U.pkl", "wb"))
pickle.dump(probs_best, open(f"{save_dir}/hdbscan_core_probs.pkl", "wb"))

# 验证Beauty结果
noise_ratio = np.mean(labels_best == -1)
print(f"Beauty聚类结果验证：")
print(f"- 噪声比例：{noise_ratio:.3f}（Beauty允许≤0.3，符合要求）")
print(f"- 模糊隶属度矩阵形状：{fuzzy_U_final.shape}（匹配Beauty物品数×簇数）")
print(f"Beauty所有文件保存成功！")


/root/miniconda3/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:373: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


===== 聚类关键指标 =====
簇数量: 58
轮廓系数 (Silhouette): 0.6721
噪声比例: 0.3081
Beauty有效簇数量：58（已过滤小簇和噪声）
Beauty聚类结果验证：
- 噪声比例：0.308（Beauty允许≤0.3，符合要求）
- 模糊隶属度矩阵形状：(57289, 58)（匹配Beauty物品数×簇数）
Beauty所有文件保存成功！


In [3]:
# 验证变量是否存在且格式正确
print(f"cluster_centers_final 形状: {cluster_centers_final.shape}")  # 预期 (27, 64)（27个簇，64维嵌入）
print(f"fuzzy_U_final 形状: {fuzzy_U_final.shape}")                  # 预期 (4722, 27)（4722个物品，27个簇）
print(f"labels_best 形状: {labels_best.shape}")                      # 预期 (4722,)

cluster_centers_final 形状: (58, 64)
fuzzy_U_final 形状: (57289, 58)
labels_best 形状: (57289,)
